In [41]:
import torch
import os
from torchvision import datasets, transforms
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.0,), (1,))
])
data="./data_mnsit"
device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
trainset = datasets.MNIST(root=data, train=True, download=True, transform=transform)
testset = datasets.MNIST(root=data, train=False, download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(dataset=trainset, batch_size=64, shuffle=True)
testloader = torch.utils.data.DataLoader(dataset=testset, batch_size=64, shuffle=False)

In [42]:
print(len(trainset),len(testset))
print(len(trainloader),len(testloader))

60000 10000
938 157


In [43]:
class MLP(torch.nn.Module):
    def __init__(self):
        super(MLP, self).__init__()
        self.fc1 = torch.nn.Linear(28*28, 16)
        self.fc2 = torch.nn.Linear(16, 16)
        self.fc3 = torch.nn.Linear(16, 10)

    def forward(self, x):
        x = x.view(-1, 28*28)
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)
        return x
model = MLP()

In [44]:
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [45]:
checkpoint_dir = 'checkpoints_mlp_msnit'
os.makedirs(checkpoint_dir, exist_ok=True)

def find_last_checkpoint():
    checkpoints = [f for f in os.listdir(checkpoint_dir) if f.startswith('model_epoch_')]
    if not checkpoints:
        return None, 0
    checkpoints.sort(key=lambda x: int(x.split('_')[-1].split('.')[0]), reverse=True)
    last_checkpoint = checkpoints[0]
    last_epoch = int(last_checkpoint.split('_')[-1].split('.')[0])
    return os.path.join(checkpoint_dir, last_checkpoint), last_epoch

In [46]:
def evaluate_model(testloader):
    checkpoint_path, start_epoch = find_last_checkpoint()
    if checkpoint_path:
        print(f"Loading latest checkpoint from {checkpoint_path}")
        model.load_state_dict(torch.load(checkpoint_path))
    else:
        print("No checkpoint found. Evaluating with the current model state.")
        return "No checkpoints"
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in testloader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    accuracy = 100 * correct / total
    print(f"Accuracy: {accuracy}%")
    return accuracy

In [47]:
device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
model.to(device)
def train_model(model, trainloader, criterion, optimizer, num_epochs=10):
    checkpoint_path, start_epoch = find_last_checkpoint()
    if checkpoint_path:
        print(f"Resuming training from {checkpoint_path}")
        model.load_state_dict(torch.load(checkpoint_path))
    else:
        print("No checkpoint found. Starting from scratch.")
        start_epoch = 0
    for epoch in range(start_epoch, num_epochs):
        running_loss = 0.0
        for inputs, labels in trainloader:
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
        checkpoint_path = os.path.join(checkpoint_dir, f'model_epoch_{epoch + 1}.pth')
        torch.save(model.state_dict(), checkpoint_path)
        print(f"Model saved at {checkpoint_path}")
        val_accuracy = evaluate_model(testloader)
        if not isinstance(val_accuracy, float):
            val_accuracy = float(val_accuracy)
        if not isinstance(running_loss, float):
            running_loss = float(running_loss)
        print(f"Epoch {epoch+1}, Validation Accuracy: {val_accuracy:.2f}%, Loss: {running_loss / len(trainloader):.4f}")
train_model(model, trainloader, criterion, optimizer, num_epochs=15)

Resuming training from checkpoints_mlp_msnit/model_epoch_15.pth


/var/folders/tq/jtgfkbln5bv4lmlbq2tvnq1h0000gn/T/ipykernel_70903/2729167981.py:7: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(checkpoint_p

In [48]:
evaluate_model(testloader)
print(model)

Loading latest checkpoint from checkpoints_mlp_msnit/model_epoch_15.pth


/var/folders/tq/jtgfkbln5bv4lmlbq2tvnq1h0000gn/T/ipykernel_70903/2950907048.py:5: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(checkpoint_p

Accuracy: 95.03%
MLP(
  (fc1): Linear(in_features=784, out_features=16, bias=True)
  (fc2): Linear(in_features=16, out_features=16, bias=True)
  (fc3): Linear(in_features=16, out_features=10, bias=True)
)
